In [56]:
from typing import List, Tuple
import difflib

def print_difference_table(baseline: str, permutations: List[Tuple[str, str]]) -> None:
    # create HtmlDiff Object
    my_html = difflib.HtmlDiff()

    # make table with problematic data, delta1 is a string
    deltas = [my_html.make_table(
        [baseline], [thing], "Baseline" , label,
    ) for  label,thing in permutations]

    html_header = '''
    <head>
        <meta http-equiv="Content-Type"
              content="text/html; charset=utf-8" />
        <title></title>
        <style type="text/css">
            td.diff_header {text-align:left}
            .diff_add {background-color:#aaffaa}
            .diff_chg {background-color:#ffff77}
            .diff_sub {background-color:#ffaaaa}
            .widthA{width:1%}
            .widthB{width:7%}
            .widthC{width:40%}
            .element_title{color: black; text-align:center}
            .element_diff{color: black; width: 96%; margin-left: auto; margin-right: auto}
        </style>
    </head>

    '''

    from IPython.display import display, HTML
    display(HTML(html_header))
    for delta in deltas:
        display(HTML(delta))

In [64]:
from datasets import load_dataset, load_from_disk
import pandas as pd
subsets = ["naive", "addition", "lexicon", "syntax"]
dataset_name = "LFrancis/GSM8k-NoOp-Plus"
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

baseline_path = dataset_name+"_main_evaluated_" + MODEL_NAME
baseline_df = pd.DataFrame( load_from_disk(baseline_path))

question_column = "question"

datasets = []
for subset in subsets:
    path = dataset_name+"_main_"+subset + "_evaluated_" + MODEL_NAME
    datasets.append(pd.DataFrame(load_from_disk(path)))

In [66]:
import random

# Randomly sample a row index
random_index = random.randint(0, len(baseline_df) - 1)

# Extract the text questions for the random index from each dataset
rows = [df.iloc[random_index][question_column] for df in datasets]
rows = list(zip(subsets, rows))
baseline_row = baseline_df.iloc[random_index][question_column]
print_difference_table(baseline_row, rows)
